In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with np.load("C:\Shared_archcraft\SML\Assignment4\mnist.npz") as data:
    x_train, y_train = data['x_train'], data['y_train']
    x_test, y_test = data['x_test'], data['y_test']

train_mask = np.isin(y_train, [0])
x_train_0 = x_train[train_mask]
y_train_0 = [0]*len(x_train_0)

train_mask = np.isin(y_train, [1])
x_train_1 = x_train[train_mask]
y_train_1 = y_train[train_mask]

train_mask = np.isin(y_train, [0, 1])
x_train_01 = x_train[train_mask]
y_train_01 = y_train[train_mask]

print(len(y_train_0))
print(len(y_train_1))


x_test_1 = x_train_1[:1000]
y_test_1 = y_train_1[:1000]
x_test_0 = x_train_0[:1000]
y_test_0 = y_train_0[:1000]

x_train_0 = x_train_0[1000:]
y_train_0 = y_train_0[1000:]
x_train_1 = x_train_1[1000:]
y_train_1 = y_train_1[1000:]

x_real_train = np.concatenate([x_train_0, x_train_1], axis=0)
y_real_train = np.concatenate([y_train_0, y_train_1], axis=0)
x_real_test = np.concatenate([x_test_0, x_test_1], axis=0)
y_real_test = np.concatenate([y_test_0, y_test_1], axis=0)

# Reshape x_real_train
# x_real_train = x_real_train.reshape(-1, 28, 28)
print(y_real_train.shape)
print(x_real_train.shape)



5923
6742
(10665,)
(10665, 28, 28)


In [3]:
def pca(x):
    X_mean = np.mean(x, axis=0)
    X_centered = x - X_mean
    X_centered_2d = X_centered.reshape(X_centered.shape[0], -1)  # Ensure X_centered is 2D
    cov_matrix = np.cov(X_centered_2d, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    p = 5
    top_indices = np.argsort(eigenvalues)[::-1][:p]
    pca_matrix = eigenvectors[:, top_indices]
    
    x_reduced = np.dot(X_centered_2d, pca_matrix)
    
    print(x_reduced.shape)
    return x_reduced

# pca(x_real_train)


In [4]:
NumOfStumps = 30

In [5]:
# residue = y_real_train.copy()
# residue = residue.astype(np.float64)



In [6]:
import numpy as np

class DecisionStump:
    def __init__(self):
        self.idx = None
        self.split_value = None
        self.left_mean = None
        self.right_mean = None
        self.min_error = float('inf')

    def train(self, X, y):
        num_samples = X.shape[0]
        num_features = X.shape[1]

        # min_error = float('inf')

        for idx in range(num_features):
            values = np.unique(X[:, idx])
            for split_value in values:
                left_indices = X[:, idx] <= split_value
                right_indices = X[:, idx] > split_value

                left_mean = np.mean(y[left_indices])
                right_mean = np.mean(y[right_indices])

                residuals = y - left_mean * left_indices.astype(int) - right_mean * right_indices.astype(int)
                error = np.sum(residuals ** 2)

                if error < self.min_error:
                    self.min_error = error
                    self.idx = idx
                    self.split_value = split_value
                    self.left_mean = left_mean
                    self.right_mean = right_mean

    def predict(self, X):
        num_samples = X.shape[0]
        predictions = np.zeros(num_samples)

        for i in range(num_samples):
            if X[i, self.idx] <= self.split_value:
                predictions[i] = self.left_mean
            else:
                predictions[i] = self.right_mean

        return predictions

class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.estimators = []

    def fit(self, X, y):
        y_pred = [np.mean(y)]*len(y)  # Initialize with the mean
        residuals = y - y_pred
        # print(y_pred)

        for _ in range(self.n_estimators):
            estimator = DecisionStump()
            estimator.train(X, residuals)
            y_pred = (y_pred + self.learning_rate * estimator.predict(X)).astype(np.float64)  # Ensure y_pred is float
            residuals = y - y_pred
            print(residuals)
            self.estimators.append(estimator)
            print(estimator.min_error)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0])

        for estimator in self.estimators:
            y_pred = (y_pred + self.learning_rate * estimator.predict(X)).astype(np.float64)  # Ensure y_pred is float

        return y_pred

# Example usage:
# model = GradientBoosting(n_estimators=100, learning_rate=0.1)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)


In [7]:
X = pca(x_real_test)
model = GradientBoosting(n_estimators=100, learning_rate=0.1)
model.fit(X, y_real_test)
# y_pred = model.predict(X_test)
# X = pca(x_real_train)
# model = GradientBoosting(n_estimators=100, learning_rate=0.1)
# model.fit(X, y_real_train)
# # y_pred = model.predict(X_test)
    

(2000, 5)


c:\Users\adars\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\adars\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[-0.4504985 -0.4504985 -0.4504985 ...  0.4502006  0.4502006  0.4502006]
6.971062739564653
[-0.40594716 -0.40594716 -0.40594716 ...  0.40538114  0.40538114
  0.40538114]
6.971062739564653
[-0.36585095 -0.36585095 -0.36585095 ...  0.36504363  0.36504363
  0.36504363]
6.971062739564653
[-0.32976436 -0.32976436 -0.32976436 ...  0.32873987  0.32873987
  0.32873987]
6.971062739564654
[-0.29728643 -0.29728643 -0.29728643 ...  0.29606648  0.29606648
  0.29606648]
6.971062739564654
[-0.26805629 -0.26805629 -0.26805629 ...  0.26666044  0.26666044
  0.26666044]
6.971062739564653
[-0.24174916 -0.24174916 -0.24174916 ...  0.240195    0.240195
  0.240195  ]
6.971062739564654
[-0.21807275 -0.21807275 -0.21807275 ...  0.2163761   0.2163761
  0.2163761 ]
6.971062739564654
[-0.19676398 -0.19676398 -0.19676398 ...  0.19493909  0.19493909
  0.19493909]
6.971062739564654
[-0.17758609 -0.17758609 -0.17758609 ...  0.17564578  0.17564578
  0.17564578]
6.971062739564654
[-0.16032598 -0.16032598 -0.16032598 ...

KeyboardInterrupt: 